In [15]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

In [16]:
data = pd.read_csv('data.csv')
top16 = data.nlargest(16, 'MEDV').index
data.drop(top16, inplace=True)
data.RM = data.RM.astype(int)

In [17]:
categorias = []

for i in data.RM.iteritems():

    valor = (i[1])
    if valor <=4:
        categorias.append('Pequeno')
    elif valor < 7:
        categorias.append('Médio')
    else:
        categorias.append('Grande')

data['categorias'] = categorias
medias_categorias = data.groupby(by='categorias')['MEDV'].mean()
dict_baseline = dict(medias_categorias)
def return_baseline(num_quartos):
    if num_quartos <= 4:
        return dict_baseline.get('Pequeno')
    elif num_quartos < 7:
        return dict_baseline.get('Médio')
    else:
        return dict_baseline.get('Grande')

In [18]:
data.head()

,Unnamed: 0,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,categorias
0,0,0.00632,18.0,2.31,0.0,0.538,6,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,Médio
1,1,0.02731,0.0,7.07,0.0,0.469,6,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,Médio
2,2,0.02729,0.0,7.07,0.0,0.469,7,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,Grande
3,3,0.03237,0.0,2.18,0.0,0.458,6,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,Médio
4,4,0.06905,0.0,2.18,0.0,0.458,7,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,Grande


> Para validar os modelos, usaremos a metrica RMSE (Erro médio quadratico), é a somatoria da diferença entre os valores preditos e os valore reais.
Ele utiliza a mesma variavel dependente. Ou seja, ele está na mesma escala das variaveis


### Preparar os dados

Separando entre dados de treino e teste.
Por convenção os dados de treino são o nosso 'x' e os dados de teste são o 'y'

In [19]:
# a variavel 'y' é o nosso alvo
y = data['MEDV']

In [20]:
# removendo colunas colineares, variavel alvo e coluna categorias
# Colunas muito correlacionadas atrapalham o modelo de previsão
X = data.drop(['RAD', 'TAX', 'MEDV','DIS', 'AGE', 'ZN', 'categorias'], axis=1)

In [21]:
X.head()

,Unnamed: 0,CRIM,INDUS,CHAS,NOX,RM,PTRATIO,B,LSTAT
0,0,0.00632,2.31,0.0,0.538,6,15.3,396.90,4.98
1,1,0.02731,7.07,0.0,0.469,6,17.8,396.90,9.14
2,2,0.02729,7.07,0.0,0.469,7,17.8,392.83,4.03
3,3,0.03237,2.18,0.0,0.458,6,18.7,394.63,2.94
4,4,0.06905,2.18,0.0,0.458,7,18.7,396.90,5.33


In [22]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

#### Separando dados de treino e teste

In [23]:
# carrega o metodo train_test_split
from sklearn.model_selection import train_test_split

In [24]:
# divide os dados entre o conjunto de treino e teste, 80% e 20% respectivamente
# define qlr valor para o parametro random_state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5) #test_size define o 'tamanho' dos dados de teste, já o random_state define a aleatoriedade do teste, é um seed

In [25]:
# impreme a qtd de linhas de cada um dos conj
print( 'X_train: numero de linhas e colunas: {}'.format(X_train.shape))
print( 'X_test: numero de linhas e colunas: {}'.format(X_test.shape))
print( 'y_train: numero de linhas e colunas: {}'.format(y_train.shape))
print( 'y_test: numero de linhas e colunas: {}'.format(y_test.shape))

X_train: numero de linhas e colunas: (392, 9)
X_test: numero de linhas e colunas: (98, 9)
y_train: numero de linhas e colunas: (392,)
y_test: numero de linhas e colunas: (98,)


### Modelo de baseline

In [26]:
X_test.head()

,Unnamed: 0,CRIM,INDUS,CHAS,NOX,RM,PTRATIO,B,LSTAT
245,245,0.19133,5.86,0.0,0.431,5,19.1,389.13,18.46
456,456,4.66883,18.10,0.0,0.713,5,20.2,10.48,19.01
75,75,0.09512,12.83,0.0,0.437,6,18.7,383.23,8.94
238,238,0.08244,4.93,0.0,0.428,6,16.6,379.41,6.36
362,362,3.67822,18.10,0.0,0.770,5,20.2,380.79,10.19


In [27]:
# definindo uma lista vazia para as predições
predicoes = []

In [29]:
# para cada elemento do teste alimenta a lista
for i in X_test.RM.iteritems():
    n_quartos = i[1]
    predicoes.append(return_baseline(n_quartos))

In [30]:
# 10 primeiras linhas da lista de predições
predicoes[:10]

[20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 35.71923076923077,
 35.71923076923077]

Atribuindo resultados a um dataframe

-   `Manteremos um log ao longo do projeto`
-   `Permite a evolução dado o baseline`

In [31]:
df_results = pd.DataFrame()

In [32]:
df_results['valor_real'] = y_test.values
df_results['valor_predito_baseline'] = predicoes
df_results.head()

,valor_real,valor_predito_baseline
0,18.5,20.130425
1,12.7,20.130425
2,21.4,20.130425
3,23.7,20.130425
4,20.8,20.130425


### Plotando o valor real e o baseline

In [33]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Valor Predito Baseline'))
# Plota a figura
fig.show()

### Avaliando o modelo de baseline

In [34]:
# Calculando a metri ca RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt

In [35]:
rmse = (np.sqrt(mean_squared_error(y_test, predicoes)))

In [37]:
# imprime a performance do modelo
print ('Performance do modelo baseline:')
print('\nRMSE é: {} '.format(rmse))
print('\nO que quer dizer que nosso modelo pode ter errado o valor {} pra cima ou pra baixo'.format(rmse))

Performance do modelo baseline:

RMSE é: 6.205816494411828 

O que quer dizer que nosso modelo pode ter errado o valor 6.205816494411828 pra cima ou pra baixo


## Machine learning
### Regressão Linear

- `Devido a correlação entre algumas features podemos usar uma regressão linear.`
- `Modelo simples e eficiente.`
- `Solução rápida e robusta.`

In [59]:
# carrega o pacote LinearRegression 
from sklearn.linear_model import LinearRegression

In [60]:
# carrega objeto do tipo LinearRegression
lin_model = LinearRegression()

In [61]:
# treina o algoritimo de regressão linear
lin_model.fit(X_train, y_train)

LinearRegression()

In [62]:
# avaliação do modelo nos dados de teste
y_pred = lin_model.predict(X_test)

In [63]:
# Calcula a metrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [64]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

Performance do modelo avaliado com os dados de teste:

RMSE é: 4.532753484786164 


In [65]:
# atribuindo os resultados ao dataframe
df_results['valor_predito_reg_linear'] = y_pred
df_results.head(10)

,valor_real,valor_predito_baseline,valor_predito_reg_linear
0,18.5,20.130425,18.705018
1,12.7,20.130425,12.497096
2,21.4,20.130425,24.224920
3,23.7,20.130425,28.429271
4,20.8,20.130425,18.240804
5,25.0,20.130425,21.591147
6,9.7,20.130425,11.514857
7,14.9,20.130425,14.666961
8,34.9,35.719231,31.189513
9,31.6,35.719231,29.577867


In [66]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Baseline'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines',
                         line = dict(color = '#FEBFB3'),
                         name='Valor Predito Regressão Linear'))

# Plota a figura
fig.show()

##### Arvore de decisão

*   `Algoritmo de aprendizado supervisionado`
*   `Explicabilidade do Modelo`
*   `Simples`

In [67]:
# importa o pct da arvore
from sklearn.tree import DecisionTreeRegressor

In [68]:
#cria o obj
regressor = DecisionTreeRegressor()

In [69]:
regressor.fit(X_train, y_train)

DecisionTreeRegressor()

### Avaliando o modelo da arvore de decisão

In [71]:
# fazendo previsões
y_pred = regressor.predict(X_test)

In [72]:
# Adicionando ao modelo
df_results['valor_predito_arvore'] = y_pred

In [73]:
df_results.head(10)

,valor_real,valor_predito_baseline,valor_predito_reg_linear,valor_predito_arvore
0,18.5,20.130425,18.705018,23.1
1,12.7,20.130425,12.497096,23.2
2,21.4,20.130425,24.224920,24.5
3,23.7,20.130425,28.429271,23.3
4,20.8,20.130425,18.240804,21.2
5,25.0,20.130425,21.591147,27.5
6,9.7,20.130425,11.514857,7.2
7,14.9,20.130425,14.666961,17.1
8,34.9,35.719231,31.189513,33.3
9,31.6,35.719231,29.577867,31.7


In [74]:
import plotly.graph_objects as go

# cria uma figura
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Valor Predito Baseline'))


# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines+markers',
                         name='Valor Predito Reg Liner'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_arvore,
                         mode='lines+markers',
                         name='Valor Predito Arvore'))
# Plota a figura
fig.show()

### Avaliação do modelo

In [75]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [76]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

Performance do modelo avaliado com os dados de teste:

RMSE é: 4.775693132725257 


#### Random Forest

- `Método Ensemble`
- `Mais robustez para os modelos`

In [77]:
# Importa o método RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

In [78]:
# cria o objeto rf_regressor
rf_regressor = RandomForestRegressor()

In [79]:
# treina o algoritmo
rf_regressor.fit(X_train, y_train)

RandomForestRegressor()

In [80]:
# avaliação do modelo nos dados de teste
y_pred = rf_regressor.predict(X_test)

In [81]:
# adiciona os resultados no dataframe df_results
df_results['valor_predito_random_forest'] = rf_regressor.predict(X_test)

In [82]:
df_results.head()

,valor_real,valor_predito_baseline,valor_predito_reg_linear,valor_predito_arvore,valor_predito_random_forest
0,18.5,20.130425,18.705018,23.1,20.613
1,12.7,20.130425,12.497096,23.2,16.480
2,21.4,20.130425,24.224920,24.5,22.848
3,23.7,20.130425,28.429271,23.3,24.888
4,20.8,20.130425,18.240804,21.2,21.851


In [83]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [84]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

Performance do modelo avaliado com os dados de teste:

RMSE é: 3.103742740627835 


In [85]:
# plota os resultados dos modelos e o valor real.
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='markers',
                         name='Valor Real'))

# Linha com os dados da baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         line = dict(color = '#FF00FF'),
                         name='Valor da Baseline'))

# Linha com os dados preditos pela arvore
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_arvore,
                         mode='lines',
                         line = dict(color = '#B2FF66'),
                         name='Valor Predito Árvore'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines',
                         line = dict(color = '#17BECF'),
                         name='Valor Predito Regressão Linear'))


# Linha com os dados preditos pela Random Forest
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_random_forest,
                         mode='lines',
                         line = dict(color = '#7F7F7F'),
                         name='Valor Predito Random Forest'))

# Plota a figura
fig.show()

### Tunning do Algoritmo de Machine Learning
**Parâmetros da Random Forest**

- `O parâmetro n_estimators define a quantidade de árvores que serão usadas para criação dos modelos. Quanto maior o número de árvores, maior será o número de modelos no ensemble.`

- `O parâmtro max_deph define a profundidade da árvore, quanto mais profunda a árvore, mais especializado nos dados de treino o nosso modelo estará.`

- `O parâmetro max_features define a quantidade máxima de nós que serão usados para separar os níveis folha da árvore`

In [86]:
X['MEDV'] = y

In [94]:
X = X.loc[:, ~X.columns.str.contains('^Unnamed')]
X.head()


,CRIM,INDUS,CHAS,NOX,RM,PTRATIO,B,LSTAT,MEDV
0,0.00632,2.31,0.0,0.538,6,15.3,396.90,4.98,24.0
1,0.02731,7.07,0.0,0.469,6,17.8,396.90,9.14,21.6
2,0.02729,7.07,0.0,0.469,7,17.8,392.83,4.03,34.7
3,0.03237,2.18,0.0,0.458,6,18.7,394.63,2.94,33.4
4,0.06905,2.18,0.0,0.458,7,18.7,396.90,5.33,36.2


In [95]:
X.to_csv('data.csv', index=False)